In [2]:
import pandas as pd

In [3]:
training = pd.read_json("FinNum_training_rebuilded.json")
dev = pd.read_json("FinNum_dev_rebuilded.json")
training

,category,id,idx,subcategory,target_num,tweet
0,[Product Number],98221616,4976,[Product Number],[334],$ARNA APD334 for Amyotrophic Lateral Sclerosis...
1,[Monetary],82321187,9839,[stop loss],[7.38],"$OCLR Noob investor that i am, put a 7.38 stop..."
2,[Product Number],103328840,1455,[Product Number],[1],$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...
3,[Product Number],104840294,1111,[Product Number],[5],$TMUS its acquisition of Layer3 TV The purchas...
4,"[Percentage, Quantity, Monetary]",69935467,2373,"[relative, Quantity, forecast]","[14, 4, 5]",$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...
5,[Temporal],94249158,1372,[date],[33],$SEED L2 Capital deal is real savvy. It takes ...
6,[Temporal],100979260,505,[date],[4],$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...
7,"[Temporal, Temporal, Temporal, Monetary]",100775772,1210,"[date, date, date, support or resistance]","[11, 09, 17., 0.96]",$WRN My fav $WRN pattern on my watchlist for 1...
8,[Temporal],100165290,7463,[date],[7],$AMD IRS still hasnt issued my 5K refund its b...
9,"[Temporal, Temporal]",102027613,7122,"[date, date]","[1990, 90]",$6E_F $DAX #BONDS Despite the biggest econ boo...


In [4]:
training.iloc[3976]["tweet"]

'$INPX I was busy with work and my order of 2900 shares @0.425 didnt go through this morning... trying to add to my 2400 shares and missed it'

In [5]:
def create_example_lines(training):
    all_words = []
    all_cat_labels = []
    all_subcat_labels = []
    subcats = set()
    cats = set()
    for row in training.iterrows():
        data = row[1]
        tweet = ' '.join(data["tweet"].split())
        tweet_words = tweet.split()
        cat_labels = ["O"]*len(tweet_words)
        subcat_labels = ["O"]*len(tweet_words)
        for i, (cat, subcat, target) in enumerate(zip(data["category"], data["subcategory"], data["target_num"])):
            loc = tweet.index(target)
            label_loc = tweet[:loc].count(' ')

            cat_labels[label_loc] = cat.lower().replace(" ", "_")
            cats.add(cat.lower().replace(" ", "_"))

            subcat_labels[label_loc] = subcat.lower().replace(" ", "_")
            subcats.add(subcat.lower().replace(" ", "_"))
        cat_labels = ' '.join(cat_labels)
        subcat_labels = ' '.join(subcat_labels)
        all_words.append(tweet)
        all_cat_labels.append(cat_labels)
        all_subcat_labels.append(subcat_labels)
    print(cats)
    print(subcats)
    return all_words, all_cat_labels, all_subcat_labels

In [6]:
all_words, all_cat_labels, all_subcat_labels = create_example_lines(training)
train_dev = pd.DataFrame({"cat_labels": all_cat_labels,
              "subcat_labels": all_subcat_labels,
              "words": all_words})

all_words, all_cat_labels, all_subcat_labels = create_example_lines(dev)
test = pd.DataFrame({"cat_labels": all_cat_labels,
              "subcat_labels": all_subcat_labels,
              "words": all_words})


{'percentage', 'option', 'quantity', 'monetary', 'product_number', 'indicator', 'temporal'}
{'maturity_date', 'support_or_resistance', 'forecast', 'money', 'absolute', 'stop_loss', 'date', 'quantity', 'quote', 'relative', 'product_number', 'exercise_price', 'indicator', 'buy_price', 'time', 'sell_price', 'change'}
{'percentage', 'quantity', 'option', 'product_number', 'monetary', 'indicator', 'temporal'}
{'maturity_date', 'money', 'forecast', 'support_or_resistance', 'stop_loss', 'date', 'quantity', 'quote', 'relative', 'product_number', 'exercise_price', 'indicator', 'change', 'buy_price', 'time', 'sell_price', 'absolute'}


In [7]:
print(len(train_dev))
train_len = int(len(train_dev) * .9)
print(train_len)
train = train_dev[:train_len]
dev = train_dev[train_len:]
print(train.shape)
print(dev.shape)

3993
3593
(3593, 3)
(400, 3)


In [8]:

train.to_csv("train.csv")
dev.to_csv("dev.csv")
test.to_csv("test.csv")


In [11]:
cats = {'monetary', 'option', 'percentage', 'indicator', 'quantity', 'temporal', 'product_number'}
cats

{'indicator',
 'monetary',
 'option',
 'percentage',
 'product_number',
 'quantity',
 'temporal'}

In [12]:
subcats = {'absolute', 'date', 'quote', 'relative', 'quantity', 'indicator', 'maturity_date', 'forecast', 'exercise_price', 'support_or_resistance', 'product_number', 'stop_loss', 'change', 'money', 'time', 'buy_price', 'sell_price'}
subcats

{'absolute',
 'buy_price',
 'change',
 'date',
 'exercise_price',
 'forecast',
 'indicator',
 'maturity_date',
 'money',
 'product_number',
 'quantity',
 'quote',
 'relative',
 'sell_price',
 'stop_loss',
 'support_or_resistance',
 'time'}